In [ ]:
pip install rul-adapt

In [ ]:
import rul_datasets
import rul_adapt
import pytorch_lightning as pl
import omegaconf

In [ ]:
pl.seed_everything(42, workers=True)  # makes is reproducible
pre_training, main_training = rul_adapt.construct.get_adarul(
    3, 1, {"max_epochs": 1}, {"max_epochs": 1}
)

INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
pre_dm, pre_approach, pre_trainer = pre_training
pre_trainer.fit(pre_approach, pre_dm)

INFO:pytorch_lightning.callbacks.model_summary:
  | Name               | Type                     | Params
----------------------------------------------------------------
0 | train_loss         | MeanSquaredError         | 0     
1 | val_loss           | MeanSquaredError         | 0     
2 | test_loss          | MeanSquaredError         | 0     
3 | evaluator          | AdaptionEvaluator        | 0     
4 | _feature_extractor | ActivationDropoutWrapper | 62.5 K
5 | _regressor         | FullyConnectedHead       | 6.3 K 
----------------------------------------------------------------
68.7 K    Trainable params
0         Non-trainable params
68.7 K    Total params
0.275     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
dm, approach, domain_disc, trainer = main_training
approach.set_model(pre_approach.feature_extractor, pre_approach.regressor, domain_disc)
trainer.fit(approach, dm)
trainer.test(approach, dm)

INFO:pytorch_lightning.callbacks.model_summary:
  | Name                     | Type                     | Params
----------------------------------------------------------------------
0 | gan_loss                 | BCEWithLogitsLoss        | 0     
1 | evaluator                | AdaptionEvaluator        | 0     
2 | _feature_extractor       | ActivationDropoutWrapper | 62.5 K
3 | _regressor               | FullyConnectedHead       | 6.3 K 
4 | _domain_disc             | FullyConnectedHead       | 6.3 K 
5 | frozen_feature_extractor | ActivationDropoutWrapper | 62.5 K
----------------------------------------------------------------------
75.0 K    Trainable params
62.5 K    Non-trainable params
137 K     Total params
0.550     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (36) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     test/source/rmse      │    24.637426376342773     │                           │
│     test/source/score     │    1309.6790928335834     │                           │
│     test/target/rmse      │                           │     31.75823402404785     │
│     test/target/score     │                           │      2989.0340537855      │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

[{'test/source/rmse/dataloader_idx_0': 24.637426376342773,
  'test/source/score/dataloader_idx_0': 1309.6790928335834},
 {'test/target/rmse/dataloader_idx_1': 31.75823402404785,
  'test/target/score/dataloader_idx_1': 2989.0340537855}]

In [ ]:
three2one_config = rul_adapt.construct.get_adarul_config(3, 1)
print(omegaconf.OmegaConf.to_yaml(three2one_config, resolve=True))

dm:
  source:
    _target_: rul_datasets.CmapssReader
    fd: 3
    window_size: 30
    max_rul: 130
    operation_condition_aware_scaling: true
  target:
    fd: 1
    percent_broken: 1.0
  batch_size: 10
feature_extractor:
  _convert_: all
  _target_: rul_adapt.model.ActivationDropoutWrapper
  wrapped:
    _target_: rul_adapt.model.LstmExtractor
    input_channels: 14
    units:
    - 32
    - 32
    - 32
    bidirectional: true
  dropout: 0.5
regressor:
  _convert_: all
  _target_: rul_adapt.model.FullyConnectedHead
  input_channels: 64
  act_func_on_last_layer: false
  units:
  - 64
  - 32
  - 1
  dropout: 0.5
domain_disc:
  _convert_: all
  _target_: rul_adapt.model.FullyConnectedHead
  input_channels: 64
  act_func_on_last_layer: false
  units:
  - 64
  - 32
  - 1
adarul_pre:
  _target_: rul_adapt.approach.SupervisedApproach
  lr: 0.0001
  loss_type: mse
  optim_type: adam
  rul_scale: 130
adarul:
  _target_: rul_adapt.approach.AdaRulApproach
  lr: 0.0001
  max_rul: 130
  num_dis

In [ ]:
import pandas as pd

# Load your source dataset
source_dataset = pd.read_csv('source_dataset.csv')

# Load your target dataset
target_dataset = pd.read_csv('target_dataset.csv')

# Extract features and labels from the datasets
X_source = source_dataset[['feature1', 'feature2', ...]]  # Replace feature1, feature2, ... with actual feature names
y_source = source_dataset['RUL']  # Assuming 'RUL' is the column containing labels

X_target = target_dataset[['feature1', 'feature2', ...]]  # Replace feature1, feature2, ... with actual feature names
y_target = target_dataset['RUL']  # Assuming 'RUL' is the column containing labels


In [ ]:
source = rul_datasets.CmapssReader(3)
target = source.get_compatible(1, percent_broken=0.8)

pre_dm = rul_datasets.RulDataModule(source, batch_size=32)
dm = rul_datasets.DomainAdaptionDataModule(
    pre_dm, rul_datasets.RulDataModule(target, batch_size=32),
)

feature_extractor = rul_adapt.model.CnnExtractor(
    input_channels=14,
    units=[16, 16, 16],
    seq_len=30,
    fc_units=8,
)
regressor = rul_adapt.model.FullyConnectedHead(
    input_channels=8,
    units=[8, 1],
    act_func_on_last_layer=False,
)
domain_disc = rul_adapt.model.FullyConnectedHead(
    input_channels=8,
    units=[8, 1],
    act_func_on_last_layer=False,
)

pre_approach = rul_adapt.approach.SupervisedApproach(
    lr=0.001, loss_type="mse", optim_type="adam", rul_scale=source.max_rul
)
pre_approach.set_model(feature_extractor, regressor)
pre_trainer = pl.Trainer(max_epochs=1)
trainer.fit(pre_approach, pre_dm)

approach = rul_adapt.approach.AdaRulApproach(
    lr=0.001,
    max_rul=source.max_rul,
    num_disc_updates=35,
    num_gen_updates=1,
)
approach.set_model(
    pre_approach.feature_extractor, pre_approach.regressor, domain_disc
)
trainer = pl.Trainer(max_epochs=1)
trainer.fit(approach, dm)
trainer.test(approach, dm)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /content/lightning_logs/version_1/checkpoints exists and is not empty.
INFO:pytorch_lightning.callbacks.model_summary:
  | Name               | Type               | Params
----------------------------------------------------------
0 | train_loss         | MeanSquaredError   | 0     
1 | val_loss           | MeanSquaredError   | 0     
2 | test_loss          | MeanSquaredError   | 0     
3 | evaluator          | AdaptionEvaluator  | 0     
4 | _feature_extractor | CnnExtractor       | 5.3 K 
5 | _regressor         | FullyConnectedHead | 81    
------------------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (36) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                     | Type               | Params
----------------------------------------------------------------
0 | gan_loss                 | BCEWithLogitsLoss  | 0     
1 | evaluator                | AdaptionEvaluator  | 0     
2 | _feature_extractor      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     test/source/rmse      │     65.22634887695312     │                           │
│     test/source/score     │     70071.39300376156     │                           │
│     test/target/rmse      │                           │     66.3141860961914      │
│     test/target/score     │                           │     64524.29218847218     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

[{'test/source/rmse/dataloader_idx_0': 65.22634887695312,
  'test/source/score/dataloader_idx_0': 70071.39300376156},
 {'test/target/rmse/dataloader_idx_1': 66.3141860961914,
  'test/target/score/dataloader_idx_1': 64524.29218847218}]

In [ ]:
#obdreader instead of cmapssreader
import os
import tempfile
import warnings
import zipfile
from typing import Union, List, Tuple, Dict, Optional

import numpy as np
from sklearn import preprocessing as scalers  # type: ignore

from rul_datasets.reader import scaling
from rul_datasets.reader.data_root import get_data_root
from rul_datasets.reader.abstract import AbstractReader
from rul_datasets import utils


CMAPSS_URL =" /content/dev_FD002.txt"


class obdReader(rul_datasets.reader.cmapss.CmapssReader):
    """
    This reader represents the NASA CMAPSS Turbofan Degradation dataset. Each of its
    four sub-datasets contains a training and a test split. Upon first usage,
    the training split will be further divided into a development and a validation
    split. 20% of the original training split is reserved for validation.

    The features are provided as sliding windows over each time series in the
    dataset. The label of a window is the label of its last time step. The RUL labels
    are capped by a maximum value. The original data contains 24 channels per time
    step. Following the literature, we omit the constant channels and operation
    condition channels by default. Therefore, the default channel indices are 4, 5,
    6, 9, 10, 11, 13, 14, 15, 16, 17, 19, 22 and 23.

    The features are min-max scaled between -1 and 1. The scaler is fitted on the
    development data only.

    Examples:
        Default channels
        >>> import rul_datasets
        >>> fd1 = rul_datasets.reader.CmapssReader(fd=1, window_size=30)
        >>> fd1.prepare_data()
        >>> features, labels = fd1.load_split("dev")
        >>> features[0].shape
        (163, 30, 14)

        Custom channels
        >>> import rul_datasets
        >>> fd1 = rul_datasets.reader.CmapssReader(fd=1, feature_select=[1, 2, 3])
        >>> fd1.prepare_data()
        >>> features, labels = fd1.load_split("dev")
        >>> features[0].shape
        (163, 30, 3)
    """

    _FMT: str = (
        "%d %d %.4f %.4f %.1f %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f "
        "%.2f %.2f %.2f %.2f %.2f %.2f %.4f %.2f %d %d %.2f %.2f %.4f"
    )
    _TRAIN_PERCENTAGE: float = 0.8
    _WINDOW_SIZES: Dict[int, int] = {1: 30, 2: 20, 3: 30, 4: 15}
    _DEFAULT_CHANNELS: List[int] = [4, 5, 6, 9, 10, 11, 13, 14, 15, 16, 17, 19, 22, 23]
    _NUM_TRAIN_RUNS: Dict[int, int] = {1: 80, 2: 208, 3: 80, 4: 199}
    _CONDITION_BOUNDARIES: List[Tuple[float, float]] = [
        (-0.009, 0.009),  # Different from paper to include FD001 and FD003
        (9.998, 10.008),
        (19.998, 20.008),
        (24.998, 25.008),
        (34.998, 35.008),
        (41.998, 42.008),
    ]
    _CONDITION_COLUMN: int = 0
    _CMAPSS_ROOT: str = os.path.join(get_data_root(), "CMAPSS")

    def __init__(
        self,
        fd: int,
        window_size: Optional[int] = None,
        max_rul: Optional[int] = 125,
        percent_broken: Optional[float] = None,
        percent_fail_runs: Optional[Union[float, List[int]]] = None,
        feature_select: Optional[List[int]] = None,
        truncate_val: bool = False,
        operation_condition_aware_scaling: bool = False,
        truncate_degraded_only: bool = False,
    ) -> None:
        """
        Create a new CMAPSS reader for one of the sub-datasets. The maximum RUL value
        is set to 125 by default. The 14 feature channels selected by default can be
        overridden by passing a list of channel indices to `feature_select`. The
        default window size is defined per sub-dataset as the minimum time series
        length in the test set.

        The data can be scaled separately for each operation condition, as done by
        Ragab et al. This only affects FD002 and FD004 due to them having multiple
        operation conditions.

        For more information about using readers, refer to the [reader]
        [rul_datasets.reader] module page.

        Args:
            fd: Index of the selected sub-dataset
            window_size: Size of the sliding window. Default defined per sub-dataset.
            max_rul: Maximum RUL value of targets.
            percent_broken: The maximum relative degradation per time series.
            percent_fail_runs: The percentage or index list of available time series.
            feature_select: The index list of selected feature channels.
            truncate_val: Truncate the validation data with `percent_broken`, too.
            operation_condition_aware_scaling: Scale data separatly for each
                                               operation condition.
            truncate_degraded_only: Only truncate the degraded part of the data
                                    (< max RUL).
        """
        super().__init__(
            fd,
            window_size,
            max_rul,
            percent_broken,
            percent_fail_runs,
            truncate_val,
            truncate_degraded_only,
        )
        # Select features according to https://doi.org/10.1016/j.ress.2017.11.021
        if feature_select is None:
            feature_select = self._DEFAULT_CHANNELS
        self.feature_select = feature_select
        self.operation_condition_aware_scaling = operation_condition_aware_scaling

    @property
    def dataset_name(self) -> str:
        return "cmapss"

    @property
    def fds(self) -> List[int]:
        """Indices of available sub-datasets."""
        return list(self._WINDOW_SIZES)

    def default_window_size(self, fd: int) -> int:
        return self._WINDOW_SIZES[fd]

    def prepare_data(self) -> None:
        """
        Prepare the CMAPSS dataset. This function needs to be called before using the
        dataset for the first time.

        The dataset is downloaded from a custom mirror and extracted into the data
        root directory. The training data is then split into development and
        validation set. Afterwards, a scaler is fit on the development features.
        Previously completed steps are skipped.
        """
        if not os.path.exists(self._CMAPSS_ROOT):
            _download_cmapss(get_data_root())
        # Check if training data was already split
        if not os.path.exists(self._get_feature_path("dev")):
            warnings.warn(
                f"Training data for FD{self.fd:03d} not "
                f"yet split into dev and val. Splitting now."
            )
            self._split_fd_train(self._get_feature_path("train"))
        if not os.path.exists(self._get_scaler_path()):
            self._prepare_scaler()

    def _prepare_scaler(self) -> None:
        dev_features, ops_cond = self._load_features(self._get_feature_path("dev"))
        dev_features, _ = self._split_time_steps_from_features(dev_features)
        scaler = self._fit_scaler(dev_features, ops_cond)
        scaling.save_scaler(scaler, self._get_scaler_path())

    def _fit_scaler(self, features, operation_conditions):
        scaler = scalers.MinMaxScaler(feature_range=(-1, 1))
        if self.operation_condition_aware_scaling:
            scaler = scaling.OperationConditionAwareScaler(
                scaler, self._CONDITION_BOUNDARIES
            )
            operation_conditions = [
                c[:, self._CONDITION_COLUMN] for c in operation_conditions
            ]
            scaler = scaling.fit_scaler(features, scaler, operation_conditions)
        else:
            scaler = scaling.fit_scaler(features, scaler)

        return scaler

    def _split_fd_train(self, train_path: str) -> None:
        train_data = np.loadtxt(train_path)

        # Split into runs
        _, samples_per_run = np.unique(train_data[:, 0], return_counts=True)
        split_indices = np.cumsum(samples_per_run)[:-1]
        split_train_data = np.split(train_data, split_indices, axis=0)

        split_idx = int(len(split_train_data) * self._TRAIN_PERCENTAGE)
        dev_data = np.concatenate(split_train_data[:split_idx])
        val_data = np.concatenate(split_train_data[split_idx:])

        data_root, train_file = os.path.split(train_path)
        dev_file = train_file.replace("train_", "dev_")
        dev_file = os.path.join(data_root, dev_file)
        np.savetxt(dev_file, dev_data, fmt=self._FMT)  # type: ignore
        val_file = train_file.replace("train_", "val_")
        val_file = os.path.join(data_root, val_file)
        np.savetxt(val_file, val_data, fmt=self._FMT)  # type: ignore

    def _get_scaler_path(self) -> str:
        return os.path.join(self._CMAPSS_ROOT, self._get_scaler_name())

    def _get_scaler_name(self) -> str:
        ops_aware = "_ops_aware" if self.operation_condition_aware_scaling else ""
        name = f"FD{self.fd:03d}_scaler_{self.feature_select}{ops_aware}.pkl"

        return name

    def _get_feature_path(self, split: str) -> str:
        return os.path.join(self._CMAPSS_ROOT, self._get_feature_name(split))

    def _get_feature_name(self, split: str) -> str:
        return f"{split}_FD{self.fd:03d}.txt"

    def load_complete_split(
        self, split: str, alias: str
    ) -> Tuple[List[np.ndarray], List[np.ndarray]]:
        file_path = self._get_feature_path(split)
        features, operation_conditions = self._load_features(file_path)
        features, time_steps = self._split_time_steps_from_features(features)
        features = self._scale_features(features, operation_conditions)

        if split in ["dev", "val"]:
            targets = self._generate_targets(time_steps)
        elif split == "test":
            targets = self._load_targets(features)
        else:
            raise ValueError(f"Unknown split {split}.")

        features, targets = self._pad_data(features, targets)

        if alias in ["dev", "val"]:
            features, targets = self._window_data(features, targets)
        elif alias == "test":
            features, targets = self._crop_data(features, targets)
        else:
            raise ValueError(f"Unknown alias {alias}.")

        return features, targets

    def _load_features(
        self, file_path: str
    ) -> Tuple[List[np.ndarray], List[np.ndarray]]:
        raw_features = np.loadtxt(file_path)

        feature_idx = [0, 1] + [idx + 2 for idx in self.feature_select]
        operation_conditions = raw_features[:, [2, 3, 4]]
        raw_features = raw_features[:, feature_idx]

        # Split into runs
        _, samples_per_run = np.unique(raw_features[:, 0], return_counts=True)
        split_idx = np.cumsum(samples_per_run)[:-1]
        features = np.split(raw_features, split_idx, axis=0)
        cond_per_run = np.split(operation_conditions, split_idx, axis=0)

        return features, cond_per_run

    def _scale_features(
        self, features: List[np.ndarray], operation_conditions: List[np.ndarray]
    ) -> List[np.ndarray]:
        scaler_path = self._get_scaler_path()
        if not os.path.exists(scaler_path):
            raise RuntimeError(
                f"Scaler for FD{self.fd:03d} with features "
                f"{self.feature_select} does not exist. "
                f"Did you call prepare_data yet?"
            )
        scaler = scaling.load_scaler(scaler_path)
        if self.operation_condition_aware_scaling:
            operation_conditions = [
                c[:, self._CONDITION_COLUMN] for c in operation_conditions
            ]
            features = scaling.scale_features(features, scaler, operation_conditions)
        else:
            features = scaling.scale_features(features, scaler)

        return features

    @staticmethod
    def _split_time_steps_from_features(
        features: List[np.ndarray],
    ) -> Tuple[List[np.ndarray], List[np.ndarray]]:
        time_steps = []
        for i, seq in enumerate(features):
            time_steps.append(seq[:, 1])
            seq = seq[:, 2:]
            features[i] = seq

        return features, time_steps

    def _generate_targets(self, time_steps: List[np.ndarray]) -> List[np.ndarray]:
        """Generate RUL targets from time steps."""
        max_rul = self.max_rul or np.inf  # no capping if max_rul is None
        targets = [np.minimum(max_rul, steps)[::-1].copy() for steps in time_steps]

        return targets

    def _load_targets(self, features: List[np.ndarray]) -> List[np.ndarray]:
        """Load target file."""
        file_name = f"RUL_FD{self.fd:03d}.txt"
        file_path = os.path.join(self._CMAPSS_ROOT, file_name)
        raw_targets = np.loadtxt(file_path)

        targets = np.split(raw_targets, len(raw_targets))
        targets = [np.arange(len(f), 0, -1) + t - 1 for f, t in zip(features, targets)]

        max_rul = self.max_rul or np.inf
        targets = [np.minimum(max_rul, t) for t in targets]

        return targets

    def _pad_data(self, features, targets):
        padded_features = []
        padded_targets = []
        for i, (feat, target) in enumerate(zip(features, targets)):
            if feat.shape[0] < self.window_size:
                warnings.warn(
                    f"Run {i} of CMAPSS FD{self.fd:03d} is shorter than window "
                    f"size {self.window_size} and will be zero-padded."
                )
                missing = self.window_size - feat.shape[0]
                feat_pad = (missing, feat.shape[1])
                feat = np.concatenate([np.zeros(feat_pad), feat])
                target = np.concatenate([np.zeros(missing), target])
            padded_features.append(feat)
            padded_targets.append(target)

        return padded_features, padded_targets

    def _window_data(
        self, features: List[np.ndarray], targets: List[np.ndarray]
    ) -> Tuple[List[np.ndarray], List[np.ndarray]]:
        """Window features with specified window size."""
        windowed_features = []
        windowed_targets = []
        for seq, target in zip(features, targets):
            windows = utils.extract_windows(seq, self.window_size)
            target = target[self.window_size - 1 :]
            windowed_features.append(windows)
            windowed_targets.append(target)

        return windowed_features, windowed_targets

    def _crop_data(
        self, features: List[np.ndarray], targets: List[np.ndarray]
    ) -> Tuple[List[np.ndarray], List[np.ndarray]]:
        """Crop length of data to specified window size."""
        cropped_features = []
        cropped_targets = []
        for seq, target in zip(features, targets):
            cropped_features.append(seq[None, -self.window_size :])
            cropped_targets.append(target[-1, None])

        return cropped_features, cropped_targets


def _download_cmapss(data_root: str) -> None:
    with tempfile.TemporaryDirectory() as tmp_path:
        print("Download CMAPSS dataset")
        download_path = os.path.join(tmp_path, "CMAPSSData.zip")
        utils.download_file(CMAPSS_URL, download_path)
        print("Extract CMAPSS dataset")
        with zipfile.ZipFile(download_path, mode="r") as f:
            f.extractall(data_root)

In [ ]:
import rul_datasets
import numpy as np
import rul_datasets
import numpy as np
import pandas as pd
import os
from rul_datasets.reader.abstract import AbstractReader

import io
import _datasource as ds


class MyReader(rul_datasets.reader.CmapssReader,AbstractReader):
    def __init__(self, dataset_path, fd, feature_select=None, **kwargs):
        self.dataset_path = dataset_path
        self.feature_select = feature_select
        # Pass other necessary arguments to parent classes
        super().__init__(fd=fd, **kwargs)

    @property
    def dataset_name(self):
        return "my_dataset"

    @property
    def fds(self):
        return [1]

    def prepare_data(self):
        pass

    def default_window_size(self, fd):
        return 30

   # def load_complete_split(self, split, alias):
        # Load your dataset from CSV file or any other source
       # data = pd.read_csv(self.dataset_path)

        # Extract features and targets from the loaded data
        #features = data.iloc[:, 1:4].values
        #targets = data.iloc[:, 5].values

        # Assuming features and targets are in the format expected by the model
        # If not, you may need to preprocess or reshape them accordingly

        #return features, targets
    def prepare_data(self) -> None:
          """
          Prepare the OBD dataset. This function needs to be called before using the
          dataset for the first time.

          The dataset is downloaded from the source and extracted into the data
          root directory. Preprocessing steps such as splitting the data and preparing
          scalers may also be performed.
          """
          if not os.path.exists(self.dataset_path):
              self._download_and_extract()
              print("OBD dataset downloaded and extracted.")


          else:
              print("Data preprocessing steps already completed.")

# Example usage:
dataset_path = "/content/dev_FD002.txt"
target= obdReader(fd=2,feature_select=[1, 2, 3, 4, 5, 6, 7, 8],window_size=30)
target.prepare_data()
features, targets = target.load_split("dev")
#print(features[0].shape)
fp = ds.open('/content/dev_FD002.txt')
#cmapss dataset with CmapssReader
source = rul_datasets.reader.CmapssReader(fd=1,window_size=30,feature_select=[1, 2, 3, 4, 5, 6, 7, 8])
source.prepare_data()
features, labels = source.load_split("dev")


In [ ]:
pre_dm = rul_datasets.RulDataModule(source, batch_size=32)
dm = rul_datasets.DomainAdaptionDataModule(
    pre_dm, rul_datasets.RulDataModule(target,batch_size=32)
)
dm.prepare_data()

/usr/local/lib/python3.10/dist-packages/rul_datasets/adaption.py:93: UserWarning: The target domain is not truncated by 'percent_broken'.This may lead to unrealistically good results.If this was intentional, please set `percent_broken` to 1.0 to silence this warning.
  warnings.warn(


In [ ]:
feature_extractor= rul_adapt.model.CnnExtractor(
    input_channels=8,
    units=[16, 16, 16],
    seq_len=30,
    fc_units=8,
)
feature_extractor_target = rul_adapt.model.CnnExtractor(
    input_channels=8,
    units=[16, 16, 16],
    seq_len=30,
    fc_units=8,
)
regressor = rul_adapt.model.FullyConnectedHead(
    input_channels=8,
    units=[8, 1],
    act_func_on_last_layer=False,
)
domain_disc = rul_adapt.model.FullyConnectedHead(
    input_channels=8,
    units=[8, 1],
    act_func_on_last_layer=False,
)

pre_approach = rul_adapt.approach.SupervisedApproach(
    lr=0.001, loss_type="mse", optim_type="adam", rul_scale=source.max_rul
)
pre_approach.set_model(feature_extractor, regressor)
pre_trainer = pl.Trainer(max_epochs=1)
trainer.fit(pre_approach, pre_dm)

approach = rul_adapt.approach.AdaRulApproach(
    lr=0.001,
    max_rul=source.max_rul,
    num_disc_updates=35,
    num_gen_updates=1,
)
approach.set_model(
    pre_approach.feature_extractor, pre_approach.regressor, domain_disc
)
trainer = pl.Trainer(max_epochs=1)
trainer.fit(approach, dm)
trainer.test(approach, dm)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name               | Type               | Params
----------------------------------------------------------
0 | train_loss         | MeanSquaredError   | 0     
1 | val_loss           | MeanSquaredError   | 0     
2 | test_loss          | MeanSquaredError   | 0     
3 | evaluator          | AdaptionEvaluator  | 0     
4 | _feature_extractor | CnnExtractor       | 5.0 K 
5 | _regressor         | FullyConnectedHead | 81    
----------------------------------------------------------
5.1 K     Trainable params
0         Non-trainable params
5.1 K     Total params
0.021     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
<ipython-input-100-2cd26aab55de>:320: UserWarning: Run 27 of CMAPSS FD002 is shorter than window size 30 and will be zero-padded.
  warnings.warn(
<ipython-input-100-2cd26aab55de>:320: UserWarning: Run 58 of CMAPSS FD002 is shorter than window size 30 and will be zero-padded.
  warnings.warn(
<ipython-input-100-2cd26aab55de>:320: UserWarning: Run 82 of CMAPSS FD002 is shorter than window size 30 and will be zero-padded.
  warnings.warn(
<ipython-input-100-2cd26aab55de>:320: UserWarning: Run 121 of CMAPSS FD002 is shorter than window size 30 and will be zero-padded.
  warnings.warn(
<i

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
<ipython-input-100-2cd26aab55de>:320: UserWarning: Run 27 of CMAPSS FD002 is shorter than window size 30 and will be zero-padded.
  warnings.warn(
<ipython-input-100-2cd26aab55de>:320: UserWarning: Run 58 of CMAPSS FD002 is shorter than window size 30 and will be zero-padded.
  warnings.warn(
<ipython-input-100-2cd26aab55de>:320: UserWarning: Run 82 of CMAPSS FD002 is shorter than window size 30 and will be zero-padded.
  warnings.warn(
<ipython-input-100-2cd26aab55de>:320: UserWarning: Run 121 of CMAPSS FD002 is shorter than window size 30 and will be zero-padded.
  warnings.warn(
<ipython-input-100-2cd26aab55de>:320: UserWarning: Run 200 of CMAPSS FD002 is shorter than window size 30 and will be zero-padded.
  warnings.warn(
<ipython-input-100-2cd26aab55de>:320: UserWarning: Run 222 of CMAPSS FD002 is shorter than window size 30 and will be zero-padded.
  warnings.warn(


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃       DataLoader 1        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│     test/source/rmse      │     40.42460250854492     │                           │
│     test/source/score     │     7188.324123533159     │                           │
│     test/target/rmse      │                           │    43.180625915527344     │
│     test/target/score     │                           │    24635.392110552784     │
└───────────────────────────┴───────────────────────────┴───────────────────────────┘

[{'test/source/rmse/dataloader_idx_0': 40.42460250854492,
  'test/source/score/dataloader_idx_0': 7188.324123533159},
 {'test/target/rmse/dataloader_idx_1': 43.180625915527344,
  'test/target/score/dataloader_idx_1': 24635.392110552784}]

In [ ]:
import rul_datasets
fd1 = rul_datasets.reader.CmapssReader(fd=1,feature_select=[1, 2, 3, 4, 5, 6, 7, 8])
fd2 =MyReader(dataset_path,fd=2,feature_select=[1, 2, 3, 4, 5, 6, 7, 8])
source = rul_datasets.RulDataModule(fd1, 32)
target = rul_datasets.RulDataModule(fd2, 32)
dm = rul_datasets.DomainAdaptionDataModule(source, target)
dm.prepare_data()
dm.setup()
train_1_2 = dm.train_dataloader()
val_1, val_2 = dm.val_dataloader()
test_1, test_2 = dm.test_dataloader()

/usr/local/lib/python3.10/dist-packages/rul_datasets/adaption.py:93: UserWarning: The target domain is not truncated by 'percent_broken'.This may lead to unrealistically good results.If this was intentional, please set `percent_broken` to 1.0 to silence this warning.
  warnings.warn(


Data preprocessing steps already completed.


FileNotFoundError: /root/.rul-datasets/CMAPSS/dev_FD002.txt not found.

In [ ]:
!pwd

/content


In [ ]:
!git remote add origin https://github.com/Emily-Essam/RUL-Estimation-OBD2.git
!git branch -M main
!git push -u origin main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git


In [ ]:
!cd /root/.rul-datasets/CMAPSS/

In [ ]:
!ls


 dev_FD002.txt	 drive	'(edited)2017-07-05_Seat_Leon_RT_S_Stau.csv'   lightning_logs   sample_data


In [ ]:
!nano dev_FD002.txt

/bin/bash: line 1: nano: command not found


In [ ]:
!ls

 _datasource.py   drive					        lightning_logs	 sample_data
 dev_FD002.txt	 '(edited)2017-07-05_Seat_Leon_RT_S_Stau.csv'   __pycache__


In [ ]:
!cd /root/.rul-datasets/CMAPSS/